In [ ]:
# df_all['ICU'] = (df_all['logons_6_mnth'] + df_all['calls_6_mnth'] + df_all['process_step']) / (df_all['clnt_tenure_mnth'] + df_all['num_accts'])
# Comparar entre los portales (1 para nuevo, 0 para antiguo)
# kpi_portal_antiguo = df_all[df_all['variation'] == 0]['ICU'].mean()
# kpi_portal_nuevo = df_all[df_all['variation'] == 1]['ICU'].mean()
# print("ICU - Portal Antiguo:", kpi_portal_antiguo)
# print("ICU - Portal Nuevo:", kpi_portal_nuevo)

# Propuesta de KPI:
# KPI: Índice de Compromiso del Usuario (ICU)
# Este KPI podría calcularse considerando varias métricas como la antigüedad del cliente, el número de cuentas, las llamadas realizadas, los inicios de sesión y el número de visitas/procesos web, diferenciando entre el portal antiguo y el nuevo (usando la columna variation).
# Fórmula sugerida:
# ICU=logons_6_mnth+calls_6_mnth+(process_step)clnt_tenure_mnth+num_accts\text{ICU} = \frac{\text{logons\_6\_mnth} + \text{calls\_6\_mnth} + (\text{process\_step})}{\text{clnt\_tenure\_mnth} + \text{num\_accts}}ICU=clnt_tenure_mnth+num_acctslogons_6_mnth+calls_6_mnth+(process_step)
# logons_6_mnth: Frecuencia de inicios de sesión.
# calls_6_mnth: Frecuencia de llamadas.
# process_step: Número de pasos completados en el portal (mayor implicación = mejor).
# clnt_tenure_mnth: Tiempo con la empresa (mayor tiempo = cliente más valioso).
# num_accts: Número de cuentas (clientes con más cuentas podrían ser más comprometidos).
# Este índice pondera el comportamiento reciente (interacciones web y llamadas) frente a la relación del cliente con la empresa (tenencia y número de cuentas).


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler



# Calcular el número de visitas por cliente
visit_counts = df_all['visit_id'].groupby(df_all['client_id']).nunique()

# Calcular el número de inicios de sesión y llamadas por cliente
logons_calls = df_all.groupby('client_id')[['logons_6_mnth', 'calls_6_mnth']].sum()

# Calcular la antigüedad del cliente
tenure = df_all.groupby('client_id')['clnt_tenure_mnth'].mean()

# Unir las métricas en un DataFrame
icu_df = pd.DataFrame({
    'visit_counts': visit_counts,
    'logons_6_mnth': logons_calls['logons_6_mnth'],
    'calls_6_mnth': logons_calls['calls_6_mnth'],
    'clnt_tenure_mnth': tenure
}).reset_index()

# Normalizar las métricas
scaler = MinMaxScaler()
icu_df[['visit_counts', 'logons_6_mnth', 'calls_6_mnth', 'clnt_tenure_mnth']] = scaler.fit_transform(
    icu_df[['visit_counts', 'logons_6_mnth', 'calls_6_mnth', 'clnt_tenure_mnth']]
)

# Calcular el ICU
icu_df['ICU'] = icu_df[['visit_counts', 'logons_6_mnth', 'calls_6_mnth', 'clnt_tenure_mnth']].mean(axis=1)

icu_df[['client_id', 'ICU']]


In [ ]:
# import matplotlib.pyplot as plt

# # Divide el DataFrame en función de la columna "variation"
# df_ambos_portales_grouped = df_ambos_portales.groupby('variation')

# # Crea un gráfico de barras para cada grupo
# for name, group in df_ambos_portales_grouped:
#     group['process_step'].value_counts().plot(kind='bar', alpha=0.5, color='blue', label=name)

# # Agrega título y etiquetas a los ejes
# plt.title('Número de veces que se repite cada valor en la columna "process_step"')
# plt.xlabel('Valor de "process_step"')
# plt.ylabel('Número de veces')
# plt.legend()

# # Muestra el gráfico
# plt.show()